In [ ]:
import z5py
import numpy
import matplotlib.pyplot as plt

import torch
from pathlib import Path
from functools import partial
from tifffile import imread

import sys

sys.path.append("..")

from lnet.utils.metrics import MSSSIM, SSIM, NRMSE, PSNR

In [ ]:
# ls_fname = "F02-FishStaticLS_488nm_8f43dd13abf527f124bc0463bd5638ed2754dbe471a80a6c5867a675.n5"
# rl_fname = "F02-FishStaticLS_488nm_e555755a6af77c8843c7871a6e3947f6f00ff9d4bdbd0effe0e3fde9.n5"
# data_path = "/g/kreshuk/beuttenm/repos/lensletnet/data/"
ls_fname = "fish/02_LS1r/0/19-09-30_17-28_5f06630_r_regional_c"
rl_fname = "fish/02_LS1r/1-lr/19-09-30_18-11_72e86b7_rfishrl_c"
data_path = "/g/kreshuk/beuttenm/repos/lensletnet/logs/"


In [ ]:
# path = data_path + ls_fname
# assert Path(path).exists(), path
# file = z5py.File(path, mode="r")
# _, ls = file["x"], file["y"][:4]

# path = data_path + rl_fname
# assert Path(path).exists(), path
# file = z5py.File(path, mode="r")
# _, rl = file["x"], file["y"][:4]

def get_stack(path):
    paths = sorted(list(path.glob("*.tif")))
    ret = []
    for path in paths:
        ret.append((imread(path.as_posix()) / numpy.iinfo(numpy.uint16).max).astype(numpy.float32))
    
    return ret

ls_target = get_stack(Path(data_path + ls_fname) / "result/test/target" )
print(len(ls_target)) 

ls_pred = get_stack(Path(data_path + ls_fname) / "result/test/prediction" )
print(len(ls_pred)) 

rl_target = get_stack(Path(data_path + rl_fname) / "result/test/target" )
print(len(rl_target)) 

rl_pred = get_stack(Path(data_path + rl_fname) / "result/test/prediction" )
print(len(rl_pred)) 

In [ ]:
metric_names = "MSSSIM,SSIM,NRMSE,PSNR".split(",")
def get_metrics(tgt_stack, pred_stack):    
    metrics = [met() for met in [MSSSIM, SSIM, NRMSE, partial(PSNR, data_range=1.)]]
    for tgt, pred in zip(tgt_stack, pred_stack):
        tgt = tgt[None]
#         print('range', tgt.max(), tgt.min(), tgt.max() - tgt.min())
        pred = pred[None]
#         print(tgt.shape, pred.shape)
        for met in metrics:
            met.update((torch.from_numpy(pred), torch.from_numpy(tgt)))
        
    return [met.compute() for met in metrics]
      
for name, val in zip(metric_names, get_metrics(ls_target, ls_pred)):
    print(f"{name} ≈ {val}")

In [ ]:
for name, val in zip(metric_names, get_metrics(ls_target, rl_target)):
    print(f"{name} ≈ {val}")

In [ ]:
for name, val in zip(metric_names, get_metrics(ls_target, rl_pred)):
    print(f"{name} ≈ {val}")

In [ ]:
for name, val in zip(metric_names, get_metrics(rl_target, rl_pred)):
    print(f"{name} ≈ {val}")

In [ ]:
RL:
0.894984170794487
0.9646372944116592
0.25966364635937444
29.52416718006134

In [ ]:
lala = 3
len(f"{lala:05}")